## Análise de Indicadores da Educação
---
Coleta e análise de dados relacionados ao ensino 

In [9]:
%matplotlib inline

import requests
import pandas as pd
import numpy as np
import json
import unidecode

import matplotlib.pyplot as plt
import seaborn as sb

from bs4 import BeautifulSoup

In [10]:
# Dados
global ufs 
ufs = pd.read_csv('recursos/ufs_.csv', sep=';')
global municipios 
municipios = pd.read_csv('recursos/municipios_.csv', sep=';')
#indicadores_legenda = pd.read_csv('dados_siope/indicadores_legenda.csv', sep=';')
#indicadores = pd.read_csv('dados_siope/indicadores.csv', sep=';')

## Capturando e estruturando os dados
---

### Interpretação dos dados consolidados:
* MUNICIPIO: Descrição do municipio
* UF : Estado do registro
* ANO: Ano do registro
* QTD_ESCOLAS_MUNICIPAIS: Quantidade de escolas municipais (http://idebescola.inep.gov.br/ideb/consulta-publica)
* QTD_ESCOLAS_ESTADUAIS : Quantidade de escolas estaduais (http://idebescola.inep.gov.br/ideb/consulta-publica)
* QTD_ALUNOS_FUNDAMENTAL_1: Quantidade de alunos (SINOPSE ESTATISTICA, Fonte: http://portal.inep.gov.br/web/guest/sinopses-estatisticas-da-educacao-basica)
* QTD_ALUNOS_FUNDAMENTAL_2: Quantidade de alunos (SINOPSE ESTATISTICA, Fonte: http://portal.inep.gov.br/web/guest/sinopses-estatisticas-da-educacao-basica)
* QTD_ALUNOS_MEDIO: Quantidade de alunos (SINOPSE ESTATISTICA, Fonte: )
* ORCAMENTO_EDUCACAO : Campo RECEITA TOTAL REALIZADA do SIOPE (Consolidado de receitas - Ex.: https://www.fnde.gov.br/siope/dadosInformadosMunicipio.do?acao=pesquisar&pag=result&anos=2016&periodos=1&cod_uf=35&municipios=355030&admin=3&planilhas=124&descricaoItem=Consolidado+de+Receita&descricaodoItem=&nivel=)
* DESPESAS_EDUCACAO_PESSOAL: Campo PESSOAL E ENCARGOS SOCIAIS  do SIOPE (Consolidado de despesas - Ex.: https://www.fnde.gov.br/siope/dadosInformadosMunicipio.do?acao=pesquisar&pag=result&anos=2016&periodos=1&cod_uf=35&municipios=355030&admin=3&planilhas=125&descricaoItem=Consolidado+de+Despesa&descricaodoItem=Consolidado+de+Receita&nivel=)
* VOLUME_DISPONIVEL = Campo ORCAMENTO_EDUCAO (Planilha) - Campo DESPESAS_EDUCACAO_PESSOAL (Planilha)

In [11]:
def normalizaString(string):
    return unidecode.unidecode(string.lower())

def municipiosPorUF(codUF):
    global municipios
    lista = {}
    for idx, m in municipios[municipios["CodUF"] == codUF].iterrows():
        lista[m["Nome"]] = m["Codigo"]
    return lista
    
def codigoInep(codUF, municipio):
    cod = 0
    lMunicipios = requests.post('http://idebescola.inep.gov.br/ideb/get-municipio-by-uf', data={"pkCodEstado": codUF})
    lMunicipios = json.loads(lMunicipios.content)
    for m in lMunicipios:
        if normalizaString(m["text"]) == normalizaString(municipio):
            cod = m["value"]
            break
    return cod

def consolidacao(uf, codUF, municipio, codMunicipio, codMunicipioInep, ano):
    requests.packages.urllib3.disable_warnings()
    
    # Novo Registro Municipio
    data = {}
    data["MUNICIPIO"] = municipio
    data["UF"] = uf
    data["ANO"] = ano
    nomeMunicipio =  normalizaString(municipio)

    # Quantidade de Escolas
    page = requests.post("http://idebescola.inep.gov.br/ideb/consulta-publica", data={'pkCodEstado': codUF, 'pkCodMunicipio': codMunicipioInep})
    content = BeautifulSoup(page.content, 'html.parser')
    tbEscolas = content.find('table', class_='table-listar-escola')
    if tbEscolas:
        escolas = tbEscolas.find_all('tr', class_='coluna')
        totalMunicipal = 0
        totalEstadual = 0
        for e in range(0, len(escolas)):
            colunas = escolas[e].find_all('td')
            #for c in range(0, len(colunas)):
            #    print(colunas[c].get_text())
            # Contabilizar Escolas por Tipo (Municipal / Estadual)
            if colunas[3].get_text().lower() == 'municipal':
                totalMunicipal = totalMunicipal + 1
            elif colunas[3].get_text().lower() == 'estadual':
                totalEstadual = totalEstadual + 1
    data["QTD_ESCOLAS_MUNICIPAIS"] = totalMunicipal
    data["QTD_ESCOLAS_ESTADUAIS"] = totalEstadual

    # Quantidade de Matriculas
    matInfantil = 0
    matFundI = 0
    matFundII = 0
    if ano >= 2010: # 2008 e 2009 os dados não estão estruturados por municipio
        df = pd.read_excel('estatisticas/'+ str(ano) +'_.xlsx')
        df['UF'] = df['UF'].apply(lambda x: normalizaString(x)) # Normalizando caracteres
        df['Cidade'] = df['Cidade'].apply(lambda x: normalizaString(x)) # Normalizando caracteres
        matriculas = df.loc[ df['UF'].str.contains(normalizaString(uf)) & df['Cidade'].str.contains(nomeMunicipio) ]
        data["QTD_ALUNOS_INFANTIL"] = matriculas['Infantil_Total'].item()
        data["QTD_ALUNOS_FUNDAMENTAL_1"] = matriculas['Fundamental_I'].item()
        data["QTD_ALUNOS_FUNDAMENTAL_2"] = matriculas['Fundamental_II'].item()
        data["QTD_ALUNOS_MEDIO"] = matriculas['Médio_Total'].item()

    # Dados Financeiros        
    # Receita Total Realizada
    receitaRealizada = 0
    page = requests.get('https://www.fnde.gov.br/siope/dadosInformadosMunicipio.do?acao=pesquisar&pag=result&anos='+
                        str(ano)+'&periodos=1&cod_uf='+ str(codUF) +
                        '&municipios='+ str(codMunicipio) +
                        '&admin=3&planilhas=124&descricaoItem=Consolidado+de+Receita&descricaodoItem=&nivel=', 
                        verify=False)
    content = BeautifulSoup(page.content, 'html.parser')
    tbReceitas = content.find('table', class_='table')
    if tbReceitas:
        indicadores = tbReceitas.findAll(True, {"class":["rowA", "rowB"]})
        if indicadores:
            valores = indicadores[-1].findAll('strong')
            receitaRealizada = valores[2].get_text() # Receita Total Realizada
    receitaRealizada = float(str(receitaRealizada).replace('R$','').replace('.','').replace(',','.')) # Tratando String -> Float
    data["ORCAMENTO_EDUCACAO"] = receitaRealizada

    # Despesas com Depto. Pessoal
    despesasPessoal = 0
    page = requests.get('https://www.fnde.gov.br/siope/dadosInformadosMunicipio.do?acao=pesquisar&pag=result&anos='+
                        str(ano)+'&periodos=1&cod_uf='+ str(codUF) +
                        '&municipios='+ str(codMunicipio) +
                        '&admin=3&planilhas=125&descricaoItem=Consolidado+de+Despesa&descricaodoItem=Consolidado+de+Receita&nivel=', verify=False)
    content = BeautifulSoup(page.content, 'html.parser')
    tbDespesas = content.find('table', class_='table')
    if tbDespesas:
        indicadores = tbDespesas.findAll(True, {"class":["rowA", "rowB"]})
        if indicadores:
            valores = indicadores[1].find_all('div', class_='number')
            despesasPessoal = valores[3].get_text() # PESSOAL E ENCARGOS SOCIAIS (Despesas Pagas)
    despesasPessoal = float(str(despesasPessoal).replace('R$','').replace('.','').replace(',','.')) # Tratando String -> Float
    data["DESPESAS_EDUCACAO_PESSOAL"] = despesasPessoal

    # Volume Disponível para Investimento
    data["VOLUME_DISPONIVEL"] = data["ORCAMENTO_EDUCACAO"] - data["DESPESAS_EDUCACAO_PESSOAL"]

    return data        

In [12]:
from ipywidgets import widgets
#from IPython.display import clear_output

print("Selecione os parâmetros para a coleta de dados:")

# UF
ufDefault = 12
opts = {}
for idx, row in ufs.iterrows():
    opts[row["Nome"]] = row["Codigo"]
cbxUF = widgets.Dropdown(
    options=opts,
    value=ufDefault,
    description='UF: ',
)
def onChangeUF(change):
    if change['type'] == 'change' and change['name'] == 'value':
        #print( "changed to %s" % change['new'] )
        cbxMunicipio.options = municipiosPorUF(cbxUF.value)
cbxUF.observe(onChangeUF)
display(cbxUF)

# Municipio
opts = {"Todos os municípios": 0}
opts.update( municipiosPorUF(cbxUF.value) )
cbxMunicipio = widgets.Dropdown(
    options=opts,
    description='Município: '
)
display(cbxMunicipio)

# Ano
intervalo = list(range(2008, 2017))
opts = {"Todo o intervalo ("+str(intervalo[0])+" a "+str(intervalo[-1])+")": 0}
for ano in intervalo:
    opts[str(ano)] = ano
cbxAno = widgets.Dropdown(
    options=opts,
    value=0,
    description='Ano: ',
)
display(cbxAno)

chkSalvar = widgets.Checkbox(
    value=True,
    description='Salvar os dados consolidados (CSV)'
)
display(chkSalvar)

btnColeta = widgets.Button(
    description='Coletar Dados',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clique para iniciar a consolidação dos dados',
    icon='check'
)
def btnColeta_clicked(clk):
    global municipios
    
    ##### Consolidacao dos dados
    registros = []
    
    # Por municipio 
    if cbxMunicipio.value > 0:
        
        legMunicipios = cbxMunicipio.value # legenda
        
        # Obtendo o codigo inep
        codMunicipioInep = codigoInep(cbxUF.value, cbxMunicipio.label)

        # Por ano
        if cbxAno.value > 0:
            legAnos = cbxAno.label # legenda
            print( "Consolidando dados de ",cbxMunicipio.label,"/",cbxUF.label,"(",legAnos,"), aguarde...", end="" )
            dados = consolidacao(cbxUF.label, cbxUF.value, 
                                 cbxMunicipio.label, cbxMunicipio.value, codMunicipioInep,
                                 cbxAno.value)
            registros.append(dados)
            print("Ok!")
            
        else: # Todo o intervalo de tempo
            legAnos = str(intervalo[0])+"-"+str(intervalo[-1]) # legenda
            print( "Consolidando dados de ",cbxMunicipio.label,"/",cbxUF.label,"(",legAnos,"), aguarde...", end="" )
            for ano in intervalo:
                dados = consolidacao(cbxUF.label, cbxUF.value, 
                                     cbxMunicipio.label, cbxMunicipio.value, codMunicipioInep,
                                     ano)
                registros.append(dados)
            print("Ok!")
                
    else: # Varrer municipios
        
        legMunicipios = "Todos-Municipios" # legenda
        
        for idx, m in municipios[municipios["CodUF"] == cbxUF.value].iterrows():
            
            
            # Obtendo o codigo inep
            codMunicipioInep = codigoInep(cbxUF.value, m["Nome"])

            # Por ano
            if cbxAno.value > 0:
                legAnos = cbxAno.label # legenda
                print( "Consolidando dados de ",m["Nome"],"/",cbxUF.label,"(",legAnos,"), aguarde...", end="" )
                dados = consolidacao(cbxUF.label, cbxUF.value, 
                                     m["Nome"], m["Codigo"], codMunicipioInep,
                                     cbxAno.value)
                registros.append(dados)
                print("Ok!")

            else: # Todo o intervalo de tempo
                legAnos = str(intervalo[0])+"-"+str(intervalo[-1]) # legenda
                print( "Consolidando dados de ",m["Nome"],"/",cbxUF.label,"(",legAnos,"), aguarde...", end="" )
                for ano in intervalo:                    
                    dados = consolidacao(cbxUF.label, cbxUF.value, 
                                         m["Nome"], m["Codigo"], codMunicipioInep,
                                         ano)
                    registros.append(dados)
                print("Ok!")        
    
    print( "Processamento concluído", end="" )
    
    # Criando CSV, se habilitado
    if chkSalvar.value:
        municipios = cbxMunicipio.label
        arquivo = "dados/"+cbxUF.label+"_"+legMunicipios+"_"+legAnos+".csv"    
        dados = pd.DataFrame(registros)
        dados.to_csv(arquivo, sep=';', encoding='utf-8', index=False)
        print(", dados salvos em '"+arquivo+"'", end="")
    
    print("!")
    
btnColeta.on_click(btnColeta_clicked)
display(btnColeta)

Selecione os parâmetros para a coleta de dados:


Dropdown(description='UF: ', options={'Acre': 12, 'Alagoas': 27, 'Amapa': 16, 'Amazonas': 13, 'Bahia': 29, 'Ceara': 23, 'Distrito Federal': 53, 'Espirito Santo': 32, 'Goias': 52, 'Maranhao': 21, 'Mato Grosso': 51, 'Mato Grosso do Sul': 50, 'Minas Gerais': 31, 'Para': 15, 'Paraiba': 25, 'Parana': 41, 'Pernambuco': 26, 'Piaui': 22, 'Rio de Janeiro': 33, 'Rio Grande do Norte': 24, 'Rio Grande do Sul': 43, 'Rondonia': 11, 'Roraima': 14, 'Santa Catarina': 42, 'Sao Paulo': 35, 'Sergipe': 28, 'Tocantins': 17}, value=12)

Dropdown(description='Município: ', options={'Todos os municípios': 0, 'Acrelândia': 120001, 'Assis Brasil': 120005, 'Brasiléia': 120010, 'Bujari': 120013, 'Capixaba': 120017, 'Cruzeiro do Sul': 120020, 'Epitaciolândia': 120025, 'Feijó': 120030, 'Jordão': 120032, 'Mâncio Lima': 120033, 'Manoel Urbano': 120034, 'Marechal Thaumaturgo': 120035, 'Plácido de Castro': 120038, 'Porto Acre': 120080, 'Porto Walter': 120039, 'Rio Branco': 120040, 'Rodrigues Alves': 120042, 'Santa Rosa do Purus': 120043, 'Sena Madureira': 120050, 'Senador Guiomard': 120045, 'Tarauacá': 120060, 'Xapuri': 120070}, value=0)

Dropdown(description='Ano: ', options={'Todo o intervalo (2008 a 2016)': 0, '2008': 2008, '2009': 2009, '2010': 2010, '2011': 2011, '2012': 2012, '2013': 2013, '2014': 2014, '2015': 2015, '2016': 2016}, value=0)

Checkbox(value=True, description='Salvar os dados consolidados (CSV)')

Button(description='Coletar Dados', icon='check', style=ButtonStyle(), tooltip='Clique para iniciar a consolidação dos dados')